# Triple Attribute Based Permissions

In this notebook we want to demonstrate how to use triple attributes to limit users to viewing certain triples. To read more generally about triple attributes and their uses, go [here](https://franz.com/agraph/support/documentation/7.0.0/triple-attributes.html), and to read more about triple attributes specifically used with python, click [here](https://franz.com/agraph/support/documentation/current/python/tutorial/example017.html)

If you are connecting in a regular jupyter notebook and have activated our `ag-tutorial` environment then you do not need to run the following cell to install `agraph-python`. However if you are in a **Google Colab** please run the following cell

In [ ]:
%pip install agraph-python pycurl

We will only set the environment variables for `AGRAPH_PASSWORD`, `AGRAPH_HOST` and `AGRAPH_PORT`. If connecting to a cloud server you will not need to change the port, but please copy the host from your server. We have left a (nonworking) example in place.

In [1]:
import os
os.environ['AGRAPH_PASSWORD'] = 'GrMEKDvQFaN2bkrHJeiCbv'
os.environ['AGRAPH_HOST'] = 'https://ag197y8xsj2epl2e.allegrograph.cloud'
os.environ['AGRAPH_PORT'] = '443'

And then we import the necessary packages

In [2]:
from franz.openrdf.sail.allegrographserver import AllegroGraphServer as ag_server
from franz.openrdf.connect import ag_connect
from franz.openrdf.query.query import QueryLanguage
from franz.openrdf.repository.attributes import AttributeDefinition, TripleAttribute, UserAttribute

First, we connect as the `admin` and create a new repository called *patient-data*. We also declare the local namespace that will be used in this notebook, as well as setting the namespace in the server so the namespace can be used as a prefix for SPARQL queries. Note that the user `admin` has **superuser** privileges.

In [3]:
conn = ag_connect('patient-data', clear=True, user='admin')

f = conn.namespace('http://franz.com/')
conn.setNamespace('f', 'http://franz.com/')

We now also connect to the actual server, not just an individual repository, as the `admin`

In [4]:
server = ag_server(user='admin')

We want to create a set of triples where a subset has a 'Low Security' attribute and the rest has a 'High Security' attribute. To do this we must first set the `AttributeDefinition`, and then apply it to the *patient-data* connection. Note that in our definition of the attribute we set `ordered` to `True`, meaning that 'Low Security' is less/lower than 'High Security'. We also set a `minimum_number` value of 1 and a `maximum_number` of 1, meaning that each triple needs one and only one security attribute.

In [5]:
security = AttributeDefinition(
    name='security',
    allowed_values=['Low Security', 'High Security'],
    ordered=True,
    minimum_number=1,
    maximum_number=1)

conn.setAttributeDefinition(security)

Now that the attribute has been defined and set on the repository, we must set the attribute filter. In the following code we want to declare the following: The attribute of the triple must be *less than or equal to* the attribute set to the user. The implication of this will become more clear after the following python chunks.

In [6]:
conn.setAttributeFilter(TripleAttribute.security <= UserAttribute.security)

Now we add a set of triples with 'High Security' attributes to *patient-data*

In [7]:
triples = []
triples.append((f.subject1, f.countryCode, f.CN, None, {'security':'High Security'}))
triples.append((f.subject1, f.country, "China", None, {'security':'High Security'}))
triples.append((f.subject1, f.name, "Xin Liu", None, {'security':'High Security'}))
triples.append((f.subject1, f.streetAddress, "87 Xinghua Road", None, {'security':'High Security'}))
triples.append((f.subject1, f.city, "Beijing", None, {'security':'High Security'}))
triples.append((f.subject1, f.age, 42, None, {'security':'High Security'}))
triples.append((f.subject1, f.postal, "100007", None, {'security':'High Security'}))
conn.addTriples(triples)

And now we add a set of triples with 'Low Security' attributes.

In [8]:
triples = []
triples.append((f.subject2, f.countryCode, f.US, None, {'security':'Low Security'}))
triples.append((f.subject2, f.country, "United States of America", None, {'security':'Low Security'}))
triples.append((f.subject2, f.name, "George Smith", None, {'security':'Low Security'}))
triples.append((f.subject2, f.streetAddress, "32 Main Street", None, {'security':'Low Security'}))
triples.append((f.subject2, f.city, "Lambertville", None, {'security':'Low Security'}))
triples.append((f.subject2, f.age, 42, None, {'security':'Low Security'}))
triples.append((f.subject2, f.postal, "08554", None, {'security':'Low Security'}))
conn.addTriples(triples)

Now, we take a look at `user1` and examine their defined attributes rules for *patient-data*

### Set *Attribute Rule* in Webview to `Low Security`

Now in webview, as a `admin` you can create a new attribute rule in the *users* page for `user1`

![user1-low-security](images/user1-low-security.png)

We see that `user1` has 'Low Security' so when we connect as `user1` to *patient-data*, so they should only be able to see triples with the 'Low Security' attribute

In [9]:
conn = ag_connect('patient-data', user='user1', password='pw1')

query_string =  "select ?s ?p ?o where { ?s ?p ?o }"
with conn.executeTupleQuery(query_string) as result:
        df = result.toPandas()
df

,s,p,o
0,<http://franz.com/subject2>,<http://franz.com/countryCode>,<http://franz.com/US>
1,<http://franz.com/subject2>,<http://franz.com/country>,United States of America
2,<http://franz.com/subject2>,<http://franz.com/name>,George Smith
3,<http://franz.com/subject2>,<http://franz.com/streetAddress>,32 Main Street
4,<http://franz.com/subject2>,<http://franz.com/city>,Lambertville
5,<http://franz.com/subject2>,<http://franz.com/age>,42
6,<http://franz.com/subject2>,<http://franz.com/postal>,08554


Not only does this work with SPARQL queries, it functions similarly with any other method of querying the data. 

In [14]:
statements = conn.getStatements(None, None, None)
with statements:
    for i, statement in enumerate(statements):
        print(i, statement.getSubject(), statement.getPredicate(), statement.getObject())

0 <http://franz.com/subject2> <http://franz.com/countryCode> <http://franz.com/US>
1 <http://franz.com/subject2> <http://franz.com/country> "United States of America"
2 <http://franz.com/subject2> <http://franz.com/name> "George Smith"
3 <http://franz.com/subject2> <http://franz.com/streetAddress> "32 Main Street"
4 <http://franz.com/subject2> <http://franz.com/city> "Lambertville"
5 <http://franz.com/subject2> <http://franz.com/age> "42"^^<http://www.w3.org/2001/XMLSchema#integer>
6 <http://franz.com/subject2> <http://franz.com/postal> "08554"


### `High Security` *Attribute Rule* in Webview

Now we examine `user2` with a 'High Security' attribute rule defined.

![user2-high-security](images/user2-high-security.png)

Now we will see that since `user2` has the 'High Security' attribute, they will be able to see all triples.

In [15]:
conn = ag_connect('patient-data', user='user2', password='pw2')

query_string =  "select ?s ?p ?o where { ?s ?p ?o }"
with conn.executeTupleQuery(query_string) as result:
        df = result.toPandas()
df

,s,p,o
0,<http://franz.com/subject1>,<http://franz.com/countryCode>,<http://franz.com/CN>
1,<http://franz.com/subject1>,<http://franz.com/country>,China
2,<http://franz.com/subject1>,<http://franz.com/name>,Xin Liu
3,<http://franz.com/subject1>,<http://franz.com/streetAddress>,87 Xinghua Road
4,<http://franz.com/subject1>,<http://franz.com/city>,Beijing
5,<http://franz.com/subject1>,<http://franz.com/age>,42
6,<http://franz.com/subject1>,<http://franz.com/postal>,100007
7,<http://franz.com/subject2>,<http://franz.com/countryCode>,<http://franz.com/US>
8,<http://franz.com/subject2>,<http://franz.com/country>,United States of America
9,<http://franz.com/subject2>,<http://franz.com/name>,George Smith


In [16]:
statements = conn.getStatements(None, None, None)
with statements:
    for i, statement in enumerate(statements):
        print(i, statement.getSubject(), statement.getPredicate(), statement.getObject())

0 <http://franz.com/subject1> <http://franz.com/countryCode> <http://franz.com/CN>
1 <http://franz.com/subject1> <http://franz.com/country> "China"
2 <http://franz.com/subject1> <http://franz.com/name> "Xin Liu"
3 <http://franz.com/subject1> <http://franz.com/streetAddress> "87 Xinghua Road"
4 <http://franz.com/subject1> <http://franz.com/city> "Beijing"
5 <http://franz.com/subject1> <http://franz.com/age> "42"^^<http://www.w3.org/2001/XMLSchema#integer>
6 <http://franz.com/subject1> <http://franz.com/postal> "100007"
7 <http://franz.com/subject2> <http://franz.com/countryCode> <http://franz.com/US>
8 <http://franz.com/subject2> <http://franz.com/country> "United States of America"
9 <http://franz.com/subject2> <http://franz.com/name> "George Smith"
10 <http://franz.com/subject2> <http://franz.com/streetAddress> "32 Main Street"
11 <http://franz.com/subject2> <http://franz.com/city> "Lambertville"
12 <http://franz.com/subject2> <http://franz.com/age> "42"^^<http://www.w3.org/2001/XMLSc